In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os
zip_ref=zipfile.ZipFile('/content/drive/MyDrive/ELC-main/train_EC.zip','r')
zip_ref.extractall('/content/drive/MyDrive/ELC-main/new_data')

In [ ]:
import io
import os
from PIL import Image
import numpy as np
from numpy import mean, std
import matplotlib.pyplot as plt
from scipy import fftpack
from scipy import ndimage
from sklearn import svm, metrics
import keras
from sklearn.metrics import classification_report, confusion_matrix
import imageio
import matplotlib.image as mpimage
import cv2
import glob
import h5py
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, cross_val_predict

import datetime as dt
from six.moves import range

In [ ]:
train_path="/content/drive/MyDrive/ELC-main/new_data/train_EC"

In [ ]:
pixel_depth=255.0
train_labels=os.listdir(train_path)
test_labels=os.listdir(train_path)
train_labels.sort()
test_labels.sort()
print(train_labels)
nb_classes= 10
global_features_train=[]
train_classes=[]
test_classes=[]


i,j=0,0
k=0


['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']


In [ ]:
for training_name in train_labels:
    path=os.path.join(train_path,training_name,'*')
    files=glob.glob(path)
    current_label=training_name
    k=1
    for fl in files:
        image=(plt.imread(fl).astype(float)-pixel_depth/2)/pixel_depth
        global_feature=np.hstack([image])
        train_classes.append(current_label)
        global_features_train.append(global_feature)
        i+=1
        k+1
    print("[status] processed folder: {}".format(current_label))
    j+=1
print("[status] completed global feature extraction..")
print("[status] feature vector size {}".format(np.array(global_features_train).shape))
print("[status] training labels {}".format(np.array(train_classes).shape))


[status] processed folder: a
[status] processed folder: b
[status] processed folder: c
[status] processed folder: d
[status] processed folder: e
[status] processed folder: f
[status] processed folder: g
[status] processed folder: h
[status] processed folder: i
[status] processed folder: j
[status] completed global feature extraction..
[status] feature vector size (835, 32, 32)
[status] training labels (835,)


# New Section

In [ ]:
#labels = (np.arange(nb_classes) == labels[:,None]).astype(np.float32)
targetNames=np.unique(train_classes)
le=LabelEncoder()
target=le.fit_transform(train_classes)
print("[status] training labels encoded...")

[status] training labels encoded...


In [ ]:
n_samples, nx, ny=np.array(global_features_train).shape
d2_global_features=np.array(global_features_train).reshape((n_samples, nx*ny))
#scaler=MinMaxScaler(feature_range=(0,1))
#rescaled_features=scaler.fit_transform(d2_global_features)

In [ ]:
print("[Status] feature vector normalized...")

print("[Status] target label{}".format(target))
print("[Status] target label shape {}".format(target.shape))


[Status] feature vector normalized...
[Status] target label[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 

In [ ]:
h5f_data=h5py.File('/content/drive/MyDrive/ELC-main/new_data/data.h5','w')
h5f_data.create_dataset('dataset_1',data=np.array(d2_global_features))

h5f_label=h5py.File('/content/drive/MyDrive/ELC-main/new_data/labels.h5','w')
h5f_label.create_dataset('dataset_1',data=np.array(target))


<HDF5 dataset "dataset_1": shape (835,), type "<i8">

In [ ]:
h5f_data.close()
h5f_label.close()

In [ ]:
h5f_data=h5py.File('/content/drive/MyDrive/ELC-main/new_data/data.h5','r')
h5f_label=h5py.File('/content/drive/MyDrive/ELC-main/new_data/labels.h5','r')


In [ ]:
global_features_string_train= h5f_data['dataset_1']
global_labels_string_train=h5f_label['dataset_1']
global_features_train=np.array(global_features_string_train)
global_labels_train=np.array(global_labels_string_train)


In [ ]:
h5f_data.close()
h5f_label.close()

In [ ]:
print("[Status] training feature shape: {}".format(global_features_train.shape))

print("[Status] labels shape: {}".format(global_labels_train.shape))

[Status] training feature shape: (835, 1024)
[Status] labels shape: (835,)


In [ ]:
clf = svm.SVC(kernel='rbf')

In [ ]:
for sz in range(2,11):
  cv = KFold(n_splits=sz, random_state=1, shuffle=True)
  scores = cross_val_score(clf,global_features_train,global_labels_train , scoring='accuracy', cv=cv, n_jobs=-1)
  print('*'*50)
  print("Fold Size: ", sz)
  print('Accuracy: %.1f %%' % (mean(scores)*100))
  y_pred = cross_val_predict(clf, global_features_train,global_labels_train , cv=10)
  conf_mat = confusion_matrix(global_labels_train, y_pred)
  print("Confusion matrix:\n%s" % conf_mat)

**************************************************
Fold Size:  2
Accuracy: 82.9 %
Confusion matrix:
[[85  4  0  4  0  1  3  3  0  0]
 [ 7 86  0  4  1  0  1  0  0  1]
 [ 0  2 90  2  4  0  2  0  0  0]
 [ 7  7  1 82  0  0  1  0  1  0]
 [ 0  0 10  1 88  1  0  0  0  0]
 [ 0  2  0  1  0 47  2  0  0  2]
 [ 0  0  0  2  0  2 95  1  0  0]
 [ 6  2  1  1  1  0  3 86  0  0]
 [ 0  1  1  0  0  5  2  0  9 10]
 [ 0  0  2  3  0  2  3  0  1 43]]
**************************************************
Fold Size:  3
Accuracy: 85.5 %
Confusion matrix:
[[85  4  0  4  0  1  3  3  0  0]
 [ 7 86  0  4  1  0  1  0  0  1]
 [ 0  2 90  2  4  0  2  0  0  0]
 [ 7  7  1 82  0  0  1  0  1  0]
 [ 0  0 10  1 88  1  0  0  0  0]
 [ 0  2  0  1  0 47  2  0  0  2]
 [ 0  0  0  2  0  2 95  1  0  0]
 [ 6  2  1  1  1  0  3 86  0  0]
 [ 0  1  1  0  0  5  2  0  9 10]
 [ 0  0  2  3  0  2  3  0  1 43]]
**************************************************
Fold Size:  4
Accuracy: 85.6 %
Confusion matrix:
[[85  4  0  4  0  1  3  3  0  0]
 [ 7 